# LangGraph Agent v2 — Search + Process

Adds web search and synthesis (no enrichment step).

Quick guide:
- State: messages, search_results, optimized_query
- Nodes: analyze → search → process → END
- Observe prints: [ANALYZE], [SEARCH], [PROCESS]
- Run the last cell to see a clean, single execution trace

In [19]:
import os
from typing import List, Optional, Union, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, END

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_community.utilities import GoogleSearchAPIWrapper

load_dotenv()

True

In [20]:
class SearchResult(BaseModel):
    id: int
    title: str
    snippet: str
    link: str
    relevance_score: Optional[float] = None
    full_content: Optional[str] = None

class AgentState(BaseModel):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(description="The chat history")
    search_results: List[SearchResult] = Field(default_factory=list, description="The results from the web search")
    optimized_query: Optional[str] = Field(default=None, description="The LLM-optimized search query")
    
    class Config:
        arbitrary_types_allowed = True

In [21]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)
search_engine = GoogleSearchAPIWrapper(k=3)

In [22]:
def analyze_query(state: AgentState):
    print("[ANALYZE] Optimizing user query…")
    user_query = next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You optimize search queries. Return ONLY the optimized query."),
        ("human", "Optimize: {q}")])
    q = llm.invoke(prompt.format_messages(q=user_query)).content.strip().replace('\"','')
    return {"optimized_query": q}

In [23]:
def perform_search(state: AgentState):
    print("[SEARCH] Running web search…")
    query = state.optimized_query or next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    try:
        raw = search_engine.results(query, num_results=5)
    except Exception:
        raw = [{"title":"Search unavailable","snippet":f"Could not search: {query}","link":"https://example.com"}]
    results = [SearchResult(id=i+1,title=r.get('title','No title'),snippet=r.get('snippet',r.get('description','No content')),link=r.get('link',r.get('url','No link'))) for i,r in enumerate(raw)]
    return {"search_results": results}

In [24]:
def process_results(state: AgentState):
    print("[PROCESS] Synthesizing final answer…")
    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), "Unknown query")
    if state.search_results:
        blocks = []
        for r in state.search_results:
            blocks.append(f"Result {r.id} | {r.title}\n{r.snippet}\nSource: {r.link}")
        results_block = "\n".join(blocks)
    else:
        results_block = "No search results available."
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Use the search results to answer precisely with sources."),
        ("human", "Query: {q}\n\nResults:\n{res}")])
    ans = llm.invoke(prompt.format_messages(q=user_query, res=results_block)).content.strip()
    return {"messages": [AIMessage(content=ans)]}

In [16]:
def route_to_search(state: AgentState) -> Literal['search']:
    return 'search'
def is_done(state: AgentState) -> Literal['end']:
    return 'end'

In [ ]:
workflow_v2 = StateGraph(AgentState)
workflow_v2.add_node('analyze', analyze_query)
workflow_v2.add_node('search', perform_search)
workflow_v2.add_node('process', process_results)

agent_v2 = workflow_v2.compile()

In [ ]:
# Test v2
query = 'Who were in 2025 NBA playoffs?'
state = AgentState(messages=[HumanMessage(content=query)])
final_state = agent_v2.invoke(state)

print('==== Execution Trace ====')
print('(Watch for: [ANALYZE] → [SEARCH] → [PROCESS])')
print('\n==== Conversation ====')
for m in final_state['messages']:
    if isinstance(m, HumanMessage):
        print(f'Human: {m.content}')
    elif isinstance(m, AIMessage):
        print(f'AI: {m.content}')

==== Conversation ====
AI: Based on the search results, here are some of the teams that participated in the 2025 NBA Playoffs:

*   **Eastern Conference:** Cavaliers, Heat, Pacers, Bucks, Knicks, Pistons (Source: Result 3)
*   **Western Conference:** Oklahoma City Thunder, Memphis Grizzlies, Houston Rockets, Golden State Warriors (Source: Result 4)
*   **Play-in Tournament:** Warriors, Grizzlies, Magic, and Heat advanced from the Play-In Tournament (Source: Result 1)
